# Python Quickstart

SedonaDB for Python can be installed from [PyPI](https://pypi.org):

```shell
pip install apache-sedona[db]
```

If you can import the module and connect to a new session, you're good to go!

In [1]:
import sedonadb

sd = sedonadb.connect()
sd.sql("SELECT ST_Point(0, 1) as geom").show()

┌────────────┐
│    geom    │
│     wkb    │
╞════════════╡
│ POINT(0 1) │
└────────────┘


Most SedonaDB Python users will want to turn on interactive mode when developing code in a notebook or interactive session. Interactive mode prints results eagerly, which is usually what you want when interacting with a new data source or constructing a query. When interacting with large remote data sources or non-interactive workloads, this is usually *not* what you want; however, you can use an explicit `.show()` to force executing enough of a query to show the first few rows.

In [2]:
sedonadb.options.interactive = True
sd.sql("SELECT ST_Point(0, 1) as geom")

┌────────────┐
│    geom    │
│     wkb    │
╞════════════╡
│ POINT(0 1) │
└────────────┘

Let's create a DataFrame with one string column and one geometry column to show some of the functionality of the SedonaDB Python interface.

In [17]:
df = sd.sql("""
SELECT * FROM (VALUES 
    ('one', ST_GeomFromWkt('POINT(1 2)')), 
    ('two', ST_GeomFromWkt('POINT(3 4)')), 
    ('three', ST_GeomFromWkt('POINT(7 9)'))) 
AS t(val, point)""")

In [18]:
df.show()

┌───────┬────────────┐
│  val  ┆    point   │
│  utf8 ┆     wkb    │
╞═══════╪════════════╡
│ one   ┆ POINT(1 2) │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ two   ┆ POINT(3 4) │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ three ┆ POINT(7 9) │
└───────┴────────────┘


Verify that this object is a SedonaDB DataFrame.

In [15]:
type(df)

sedonadb.dataframe.DataFrame

Expose the DataFrame as a view and run a spatial operation on the geometry data.

In [19]:
df.to_view("fun_table")

In [24]:
sd.sql("select *, ST_Area(point) from fun_table").show()

┌───────┬────────────┬──────────────────────────┐
│  val  ┆    point   ┆ st_area(fun_table.point) │
│  utf8 ┆     wkb    ┆          float64         │
╞═══════╪════════════╪══════════════════════════╡
│ one   ┆ POINT(1 2) ┆                      0.0 │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ two   ┆ POINT(3 4) ┆                      0.0 │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ three ┆ POINT(7 9) ┆                      0.0 │
└───────┴────────────┴──────────────────────────┘
